In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cython
import timeit
import math

In [ ]:
%load_ext cython

# Native code compilation

We will see how to convert Python code to native compiled code. We will use the example of calculating the pairwise distance between a set of vectors, a $O(n^2)$ operation. 

For native code compilation, it is usually preferable to use explicit for loops and minimize the use of `numpy` vectorization and broadcasting because

- It makes it easier for the `numba` JIT to optimize
- It is easier to "cythonize"
- It is easier to port to C++

However, use of vectors and matrices is fine especially if you will be porting to use a C++ library such as Eigen.

## Timing code

### Manual

In [ ]:
import time

def f(n=1):
    start = time.time()
    time.sleep(n)
    elapsed = time.time() - start
    return elapsed

In [ ]:
f(1)

### Clock time

The `time` magic function calls the Unix `time` command. This returns 3 different times:

- user time is time spent by user code and libraries
- sys time is time spent on operating system calls (kernel calls)
- wall time is time that has elapsed from your perspective

For concurrent programs, user/sys time can be greater than wall time.

In [ ]:
%%time

time.sleep(1)

### Using `timeit`

The `-r` argument says how many runs to average over, and `-n` says how many times to run the function in a loop per run. See `%timeit?` for more information.

In [ ]:
%timeit time.sleep(0.01)

In [ ]:
%timeit -r3 time.sleep(0.01)

In [ ]:
%timeit -n10 time.sleep(0.01)

In [ ]:
%timeit -r3 -n10 time.sleep(0.01)

The `-o` flag returns an object of the time statistics

In [ ]:
t1 = %timeit -n10 -o time.sleep(0.01)

In [ ]:
t1

In [ ]:
', '.join([method for method in dir(t1) if not method.startswith('_')])

You can also use `timeit` as a Python module.

Pass it a callable with no arguments or a string. This is not as convenient as the magic function. 

In [ ]:
import timeit

In [ ]:
timeit.timeit('time.sleep(0.01)', number=10)

In [ ]:
timeit.timeit(lambda: time.sleep(0.01), number=10)

### Time unit conversions

```
1 s = 1,000 ms
1 ms = 1,000 µs
1 µs = 1,000 ns
```

## Profiling

If you want to identify bottlenecks in a Python script, do the following:
    
- First make sure that the script is modular - i.e. it consists mainly of function calls
- Each function should be fairly small and only do one thing
- Then run a profiler to identify the bottleneck function(s) and optimize them

See the Python docs on [profiling Python code](https://docs.python.org/3/library/profile.html)

Profiling can be done in a notebook with %prun, with the following readouts as column headers:

- ncalls
    - for the number of calls,
- tottime
    - for the total time spent in the given function (and excluding time made in calls to sub-functions),
- percall
    - is the quotient of tottime divided by ncalls
- cumtime
    - is the total time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
- percall
    - is the quotient of cumtime divided by primitive calls
- filename:lineno(function)
    - provides the respective data of each function 
    
See `%prun?` for more information.

In [ ]:
def foo1(n):
    return np.sum(np.square(np.arange(n)))

def foo2(n):
    return sum(i*i for i in range(n))

def foo3(n):
    [foo1(n) for i in range(10)]
    foo2(n)

def foo4(n):
    return [foo2(n) for i in range(100)]
    
def work(n):
    foo1(n)
    foo2(n)
    foo3(n)
    foo4(n)

In [ ]:
%%time

work(int(1e5))

- `-D` saves results in a form that the `pstats` moudle can parse.
- `-q` suppresses output

In [ ]:
%prun -q -D work.prof work(int(1e5))

In [ ]:
import pstats
p = pstats.Stats('work.prof')
p.print_stats()
pass

In [ ]:
p.sort_stats('time', 'cumulative').print_stats('foo')
pass

In [ ]:
p.sort_stats('ncalls').print_stats(5)
pass

We can get the results object directly.

In [ ]:
profile = %prun -r -q work(int(1e5))

In [ ]:
profile.sort_stats('cumtime').print_stats(5)
pass

We may not need `pstats` for simple analysis. This limits to calls with `foo` in the name, and sorts by `cumtime`.

In [ ]:
%prun -l foo -s cumtime work(int(1e5))

## Optimizing a function

Our example will be to optimize a function that calculates the pairwise distance between a set of vectors.

We first use a built-in function from`scipy` to check that our answers are right and also to benchmark how our code compares in speed to an optimized compiled routine.

In [ ]:
from scipy.spatial.distance import squareform, pdist

In [ ]:
n = 100
p = 100
xs = np.random.random((n, p))

We save the result to compare with our own implementations later.

In [ ]:
sol = squareform(pdist(xs))

In [ ]:
%timeit -r3 -n3 squareform(pdist(xs))

## Python

### Simple version

In [ ]:
def pdist_py(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(p):
                A[i,j] += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(A[i,j])
    return A

Note that we 

- first check that the output is **right**
- then check how fast the code is

In [ ]:
func = pdist_py
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Exploiting symmetry

In [ ]:
def pdist_sym(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            for k in range(p):
                A[i,j] += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(A[i,j])
    A += A.T
    return A

In [ ]:
func = pdist_sym
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Vectorizing inner loop

In [ ]:
def pdist_vec(xs): 
    """Vectorize inner loop."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            A[i,j] = np.sqrt(np.sum((xs[i] - xs[j])**2))
    A += A.T
    return A

In [ ]:
func = pdist_vec
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Broadcasting and vectorizing

Note that the broadcast version does twice as much work as it does not exploit symmetry.

In [ ]:
def pdist_numpy(xs):
    """Fully vectroized version."""
    return np.sqrt(np.square(xs[:, None] - xs[None, :]).sum(axis=-1))

In [ ]:
func = pdist_numpy
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 squareform(func(xs))

## JIT with `numba`

We use the `numba.jit` decorator which will trigger generation and execution of compiled code when the function is first called.

In [ ]:
from numba import jit

### Using `jit` as a function

In [ ]:
pdist_numba_py = jit(pdist_py, nopython=True, cache=True)

In [ ]:
func = pdist_numba_py
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Using `jit` as a decorator

In [ ]:
@jit(nopython=True, cache=True)
def pdist_numba_py_1(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(p):
                A[i,j] += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(A[i,j])
    return A

In [ ]:
func = pdist_numba_py_1
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Can we make the code faster?

Note that in the inner loop, we are updating a matrix when we only need to update a scalar. Let's fix this.

In [ ]:
@jit(nopython=True, cache=True)
def pdist_numba_py_2(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            d = 0.0
            for k in range(p):
                d += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(d)
    return A

In [ ]:
func = pdist_numba_py_2
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Can we make the code even faster?

We can also try to exploit symmetry.

In [ ]:
@jit(nopython=True, cache=True)
def pdist_numba_py_sym(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(d)
    A += A.T
    return A

In [ ]:
func = pdist_numba_py_sym
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Does `jit` work with vectorized code?

In [ ]:
pdist_numba_vec = jit(pdist_vec, nopython=True, cache=True)

Only inner loop vectorized

In [ ]:
%timeit -r3 -n3 pdist_vec(xs)

In [ ]:
func = pdist_numba_vec
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Does `jit` work with broadcasting?

In [ ]:
pdist_numba_numpy = jit(pdist_numpy, nopython=True, cache=True)

In [ ]:
%timeit -r3 -n3 pdist_numpy(xs)

This raises an error because `numba` does not know how to deal with dummy axes.

In [ ]:
try:
    %timeit -r3 -n3 pdist_numba_numpy(xs)
except Exception as e:
    print('Exception raised')

#### We need to use `reshape` to broadcast

In [ ]:
def pdist_numpy_(xs):
    """Fully vectroized version."""
    return np.sqrt(np.square(xs.reshape(n,1,p) - xs.reshape(1,n,p)).sum(axis=-1))

In [ ]:
pdist_numba_numpy_ = jit(pdist_numpy_, nopython=True, cache=True)

In [ ]:
%timeit -r3 -n3 pdist_numpy_(xs)

In [ ]:
func = pdist_numba_numpy_
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

### Summary

- `numba` appears to work best when converting fairly explicit Python code
- This might change in the future as the `numba` JIT compiler becomes more sophisticated
- Always check optimized code for correctness
- We can use `timeit` magic as a simple way to benchmark functions

## Cython

Cython is an Ahead Of Time (AOT) compiler. It compiles the code and replaces the function invoked with the compiled version.

In the notebook, calling `%cython -a` magic shows code colored by how many Python C API calls are being made. You want to reduce the yellow as much as possible.

In [ ]:
%%cython -a 

import numpy as np

def pdist_cython_1(xs):   
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += (xs[i,k] - xs[j,k])**2
            A[i,j] = np.sqrt(d)
    A += A.T
    return A

In [ ]:
def pdist_base(xs):   
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += (xs[i,k] - xs[j,k])**2
            A[i,j] = np.sqrt(d)
    A += A.T
    return A

In [ ]:
%timeit -r3 -n3 pdist_base(xs)

In [ ]:
func = pdist_cython_1
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

## Cython with static types

- We provide types for all variables so that Cython can optimize their compilation to C code.
- Note `numpy` functions are optimized for working with `ndarrays` and have unnecessary overhead for scalars. We therefor replace them with math functions from the C `math` library.

In [ ]:
%%cython -a 

import cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt, pow

@cython.boundscheck(False)
@cython.wraparound(False)
def pdist_cython_2(double[:, :] xs):
    cdef int n, p
    cdef int i, j, k
    cdef double[:, :] A
    cdef double d
    
    n = xs.shape[0]
    p = xs.shape[1]
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += pow(xs[i,k] - xs[j,k],2)
            A[i,j] = sqrt(d)
    for i in range(1, n):
        for j in range(i):
            A[i, j] = A[j, i]            
    return A

In [ ]:
func = pdist_cython_2
print(np.allclose(func(xs), sol))
%timeit -r3 -n1 func(xs)